In [ ]:
# PRIMEIRO PASSO

import warnings
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.core.common import SettingWithCopyWarning

In [ ]:
# PRIMEIRO PASSO

pd.options.display.max_rows = 999
pd.options.display.max_columns=999
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
diretorioCombustivel = 'C:\\Temp\\Combustiveis\\Gasolina_Etanol_Diesel_GNV\\'

dfCombustivel = pd.DataFrame()

for arquivoCombustivel in os.listdir(diretorioCombustivel):
    
    pathCombustivel = diretorioCombustivel + arquivoCombustivel
    print(pathCombustivel)
    
    df = pd.read_csv(pathCombustivel
                     , sep=';'
                     , header=0
                     , usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
                     , names=['Regiao','UF','NomMunicipio','NomRevenda','NumCNPJRevenda','NomLogradouro','NrEndereco'
                              ,'NomComplemento','NomBairro','NumCep','DscProduto','DtColeta','VlrVenda','VlrCompra','UnMedida'
                              ,'DscBandeira'
                             ]
                    ,dtype={'Regiao':'str'
                    ,'UF':'str'
                    ,'NomMunicipio':'str'
                    ,'NomRevenda':'str'
                    ,'NumCNPJRevenda':'str'
                    ,'NrEndereco':'str'
                    ,'NomLogradouro':'str'
                    ,'NomComplemento':'str'
                    ,'NomBairro':'str'
                    ,'NumCep':'str'
                    ,'DscProduto':'str'
                    ,'DtColeta':'str'
                    ,'VlrVenda':'str'
                    ,'VlrCompra':'str'
                    ,'UnMedida':'str'
                    ,'DscBandeira':'str'
                    }
                    , parse_dates=[11]
                    , dayfirst=True
                    , engine='python')
    
    dfCombustivel = dfCombustivel.append(df)

In [ ]:
# TERCEIRO PASSO

#TRATAMENTO DE VALORES NULOS    
dfCombustivel['NomComplemento'].fillna('NI',inplace=True)
dfCombustivel['NomLogradouro'].fillna('NI',inplace=True)
dfCombustivel['NomBairro'].fillna('NI',inplace=True)
dfCombustivel['VlrCompra'].fillna('0',inplace=True)

#CRIAÇÃO DE COLUNAS NOVAS DE TEMPO - ANO, MES, ANOMES
dfCombustivel['Ano'] = dfCombustivel['DtColeta'].dt.strftime('%Y')
dfCombustivel['Mes'] = dfCombustivel['DtColeta'].dt.strftime('%m')
dfCombustivel['AnoMes']  = dfCombustivel['DtColeta'].dt.strftime('%Y%m')

#CRIAÇÃO DE COLUNA ENDERECO COMPLETO 
dfCombustivel['NomEnderecoCompleto'] = dfCombustivel['NomLogradouro'] \
                                        + ', NR: ' \
                                        + dfCombustivel['NrEndereco'] \
                                        + ', COMPLEMENTO: ' \
                                        + dfCombustivel['NomComplemento'] \
                                        + ' - BAIRRO: ' \
                                        + dfCombustivel['NomBairro']

dfCombustivel['NomEnderecoCompleto'].fillna('NI',inplace=True)

#CONVERSÃO DE TIPO DE DADOS
dfCombustivel['VlrCompra'] = dfCombustivel['VlrCompra'].str.replace(',', '.').astype(float)
dfCombustivel['VlrVenda'] = dfCombustivel['VlrVenda'].str.replace(',', '.').astype(float)

In [ ]:
# QUARTO PASSO

dfCombustivel.info()

In [ ]:
# QUINTO PASSO

# APLICAÇÃO DE FILTRO PARA RETORNAR APENAS OS DADOS QUE O CLIENTE ATUA

resultEstadosAtuantes = dfCombustivel[((dfCombustivel['UF']=='CE') & (dfCombustivel['NomMunicipio']=='JUAZEIRO DO NORTE')) |
                                     ((dfCombustivel['UF']=='MA') & (dfCombustivel['NomMunicipio']=='IMPERATRIZ')) |
                                      ((dfCombustivel['UF']=='RJ') & (dfCombustivel['NomMunicipio']=='SAO GONCALO')) |
                                      ((dfCombustivel['UF']=='SP') & (dfCombustivel['NomMunicipio']=='BARUERI')) 
                                     ].groupby(['Regiao'
                                                ,'UF'
                                                ,'NomMunicipio'
                                                ,'NomBairro'
                                                ,'NomRevenda'
                                                ,'NomEnderecoCompleto'
                                                ,'DscProduto'
                                                ,'AnoMes'
                                                ,'Ano'
                                                ,'Mes'
                                                ],as_index=False).agg(VlrMedio=('VlrVenda','mean'))


In [ ]:
# SEXTO PASSO

# TRANSFORMAÇÃO DE LINHAS EM COLUNAS - COLUNA - ANOMES
# CRIANDO UMA COLUNA PARA CADA ANOMES

resultRelatorioFinal = resultEstadosAtuantes.pivot(index=['Regiao'
                                                ,'UF'
                                                ,'NomMunicipio'
                                                ,'NomBairro'
                                                ,'NomRevenda'
                                                ,'NomEnderecoCompleto'
                                                ,'DscProduto'
                                                ], columns="AnoMes", values="VlrMedio").fillna(0).reset_index()

resultRelatorioFinal

In [ ]:
# SÉTIMO PASSO

# GERAÇÃO DE UM ARQUIVO PARA CADA FILIAL 

for lista_uf in resultRelatorioFinal['UF'].unique():
    
    print('Arquivo Estado do ' + lista_uf + ' gerado com sucesso!')

    resultRelatorioFinal[resultRelatorioFinal['UF']==lista_uf].to_csv('C:\\Arquivo\\lista_{0}.csv'
                                                                      .format(lista_uf),sep=';',index=False)

In [ ]:
# OITAVO PASSO

# REPRESENTANDO A VARIAÇÃO MENSÃO NO MUNICIPIO EM GRÁFICO

reportGrafico = dfCombustivel[((dfCombustivel['UF']=='CE') & (dfCombustivel['NomMunicipio']=='JUAZEIRO DO NORTE'))
                                     ].groupby(['Regiao'
                                                ,'UF'
                                                ,'NomMunicipio'
                                                ,'DscProduto'
                                                ,'AnoMes'
                                                ,'Ano'
                                                ,'Mes'
                                                ],as_index=False).agg(VlrMedio=('VlrVenda','mean')).sort_values(by='AnoMes')


In [ ]:
# GERAÇÃO DO GRÁFICO

#criando a fig e o ax no matplotlib
fig, ax = plt.subplots(figsize=(50,10))

sns.barplot(  x = reportGrafico['AnoMes']
            , y = reportGrafico['VlrMedio']
            , hue = reportGrafico['DscProduto']
            , hue_order=reportGrafico['DscProduto'].unique()
           )

#modificação do fundo
ax.set_frame_on(False)
#adicionando um título
ax.set_title('Variação dos Preços dos Combustíveis',loc='center',pad=40,fontdict={'fontsize':40},color='#3f3f4e')
#retirando o eixo y
ax.get_yaxis().set_visible(False)
#retirnado os ticks do eixo x
ax.tick_params(axis='x',length=20,labelsize=30,colors='black')
#ajustando o título do gráfico
ax.set_xlabel('Variação Anual',labelpad=24,fontdict={'fontsize':30},color='black')

for retangulo in ax.patches:
    ax.text(retangulo.get_x() + retangulo.get_width() / 2,
          retangulo.get_height()+0.1,
          '{:.3f}'.format(float(retangulo.get_height())).replace(',','.'),
          ha ='center',
          fontsize=18
          ,color='black')
    
#plotando o gráfico
plt.show()

In [ ]:
# NONO PASSO

import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 


for lista_uf in resultRelatorioFinal['UF'].unique():
    
    print('Arquivo Estado do ' + lista_uf + ' gerado com sucesso!')

    resultRelatorioFinal[resultRelatorioFinal['UF']==lista_uf].to_csv('C:\\Arquivo\\lista_{0}.csv'
                                                                      .format(lista_uf),sep=';',index=False)


    fromaddr = "dbaassists@gmail.com"
    toaddr = "dbaassists@gmail.com"

    msg = MIMEMultipart() 
    msg['From'] = fromaddr 
    msg['To'] = toaddr 
    msg['Subject'] = "Arquivo de Variação dos Preços dos Combustíveis do Estado {0}".format(lista_uf)
    body = "Olá tudo bem? \nSegue em anexo o arquivo de Variação dos Preços dos Combustíveis.".format(lista_uf)
    msg.attach(MIMEText(body, 'plain')) 
    filename = "lista_{0}.csv".format(lista_uf)
    attachment = open("C:\Arquivo\lista_{0}.csv".format(lista_uf), "rb")
    p = MIMEBase('application', 'octet-stream') 
    p.set_payload((attachment).read()) 
    encoders.encode_base64(p) 

    p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
    msg.attach(p) 
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(fromaddr, "XXXXXXXXXXXXXXXXXXXXX") 
    text = msg.as_string() 
    s.sendmail(fromaddr, toaddr, text) 
    s.quit()    
